In [1]:
!pip install datasets lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 24.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, DatasetDict

In [ ]:
dataset = load_dataset("imdb")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
# Split the train dataset into train and validation preserving the same distribution of labels as the original one
fit_data = dataset["train"].train_test_split(
    test_size=0.2, seed=42, stratify_by_column="label"
)
train_data, val_data = fit_data["train"], fit_data["test"]
test_data = dataset["test"]

data = DatasetDict({"train": train_data, "val": val_data, "test": test_data})
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})

In [ ]:
sample = data["train"][42]
sample

{'text': 'This is a great film - esp when compared with the sometimes wearisome earnestness of today\'s politically-minded filmmakers. A film that can so easily combine sex, gender relations, politics and art is a rarity these days. While the bouyant optimism of the 1960\'s can\'t be regained, I think we can at least learn a lesson from the film\'s breezy energy and charm. I don\'t know what those who label the film "boring" were watching - there\'s so much packed into it that it never remains the same film for more that 15 min at a time.',
 'label': 1}

In [ ]:
data["train"].features["label"], data["train"].features["label"].int2str(1)

(ClassLabel(names=['neg', 'pos'], id=None), 'pos')

In [ ]:
from torch import nn
import torch
import torch.nn.functional as F
from typing import Mapping, Sequence, List, Any
import typing as ty
from collections import Counter
import re
import string

In [ ]:
def simple_tokenize(text: str) -> List[str]:
    """A simple function to tokenize a given text."""
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    tokens = text.split()
    return tokens

In [ ]:
data = data.map(lambda x: {**x, "tokenized": simple_tokenize(x["text"])})
data

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'tokenized'],
        num_rows: 20000
    })
    val: Dataset({
        features: ['text', 'label', 'tokenized'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label', 'tokenized'],
        num_rows: 25000
    })
})

In [ ]:
data["train"][42] # Now we have a new column called "tokenized" with the tokenized text

{'text': 'This is a great film - esp when compared with the sometimes wearisome earnestness of today\'s politically-minded filmmakers. A film that can so easily combine sex, gender relations, politics and art is a rarity these days. While the bouyant optimism of the 1960\'s can\'t be regained, I think we can at least learn a lesson from the film\'s breezy energy and charm. I don\'t know what those who label the film "boring" were watching - there\'s so much packed into it that it never remains the same film for more that 15 min at a time.',
 'label': 1,
 'tokenized': ['this',
  'is',
  'a',
  'great',
  'film',
  'esp',
  'when',
  'compared',
  'with',
  'the',
  'sometimes',
  'wearisome',
  'earnestness',
  'of',
  'todays',
  'politicallyminded',
  'filmmakers',
  'a',
  'film',
  'that',
  'can',
  'so',
  'easily',
  'combine',
  'sex',
  'gender',
  'relations',
  'politics',
  'and',
  'art',
  'is',
  'a',
  'rarity',
  'these',
  'days',
  'while',
  'the',
  'bouyant',
  'op

In [ ]:
def token_count(tokenized_texts: Sequence[Sequence[str]]) -> Counter[str]:
    """Count the number of times each token appears in the tokenized texts."""
    tokens = [token for text in tokenized_texts for token in text]
    token2count = Counter(tokens)
    return token2count

In [ ]:
token2count: ty.Counter = token_count(data["train"]["tokenized"])
f"Total number of unique tokens: {len(token2count)}"

'Total number of unique tokens: 107405'

In [ ]:
token2count.most_common(5)
# they are not very informative, are they? That's because we have not removed the stop words https://en.wikipedia.org/wiki/Stop_word

[('the', 268024),
 ('and', 129871),
 ('a', 129588),
 ('of', 116561),
 ('to', 108417)]

In [ ]:
# some special tokens we will use
PAD_TOKEN_STR = "<pad>" # this is used to pad the sequences to the same length
UNK_TOKEN_STR = "<unk>" # this is used to represent tokens that are not in the vocabulary

In [ ]:
# we will restrict the vocabulary to the 10k most common tokens (which is ~10% of the total number of unique tokens)
vocab_size = 10_000
vocab = {
    word: index
    for index, word in enumerate(
        [PAD_TOKEN_STR, UNK_TOKEN_STR]
        + [token for token, _ in token2count.most_common(vocab_size)]
    )
}
list(vocab.items())[:5]

[('<pad>', 0), ('<unk>', 1), ('the', 2), ('and', 3), ('a', 4)]

In [ ]:
# now we can encode the tokenized texts using the vocabulary built from the training data
data = data.map(
    lambda x: {"token_ids": torch.as_tensor([vocab.get(token, vocab[UNK_TOKEN_STR]) for token in x["tokenized"]])}
)
data.set_format(type="torch", columns=["token_ids"], output_all_columns=True)
data

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'tokenized', 'token_ids'],
        num_rows: 20000
    })
    val: Dataset({
        features: ['text', 'label', 'tokenized', 'token_ids'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label', 'tokenized', 'token_ids'],
        num_rows: 25000
    })
})

In [ ]:
data["train"][0]

{'token_ids': tensor([ 101,  872, 3603,    5,   50,  843,   43,   11,   18,   10,  837,    6,
          189,    9,   16,    4, 3686,   10, 1213,    9,   21,  294, 3015,    2,
         3686,    1,    4,    1,    2,  454,   14,  280,   10, 3144,   16,    4,
          403, 3434,  360,   17,   47, 4438,   12,  196,  541,   21,  953,    1,
         9055,    6,   49,  141,   26,   76,    4,   50,   18,   19,    2,  113,
           10,  541,   14,   38,   85,   14,   38,  728,  168,   62,   10,  207,
            4,  108,  534,   16,    2, 9275,   10, 4120,   61,    1,   27,   14,
          970,    2,   60,  630,  108,  203,  551,  127,    8,    2,  726,    5,
            2,   18,  582,   38,    2,  113,  266,   53,   50,   19,   31,  211,
           48,   14,   47,   50, 5744,  201,  361,    2, 1836,   67,   53,  352,
            3,  164,    3,  234,  190, 2491,    2,  267, 4001,   67,   32,  181,
          630,   19,   10,  629,    2, 1836,   67, 1136,   10,  794, 1327,    6,
           66, 

In [ ]:
encoding_dim: int = 100

In [ ]:
# what's the percentage of UNK tokens in each split we have by keeping only the top 10% most common tokens?
unk_id = vocab[UNK_TOKEN_STR]
for split, split_data in data.items():
    token_count = 0
    unk_count = 0
    for token_ids in split_data["token_ids"]:
        token_count += len(token_ids)
        unk_count += (token_ids == unk_id).sum()
    print(f"{split} set has {unk_count} UNK tokens over {token_count} tokens: ({unk_count / token_count:.1%})")

train set has 314022 UNK tokens over 4663531 tokens: (6.7%)
val set has 82018 UNK tokens over 1158283 tokens: (7.1%)
test set has 412969 UNK tokens over 5691098 tokens: (7.3%)


In [ ]:
class TextEncoder(nn.Module):
    def __init__(
        self,
        token2index: Mapping[str, int],
        encoding_dim: int,
        oov_token: str = UNK_TOKEN_STR,
        pad_token: str = PAD_TOKEN_STR,
        aggregation: str = "mean",
    ):
        super().__init__()
        assert pad_token in token2index
        assert oov_token in token2index

        self.token2index = token2index
        self.oov_index = self.token2index[oov_token]

        # initialize the token weights randomly
        token_weights = torch.randn(len(token2index), encoding_dim)
        token_weights[token2index[oov_token]] = token_weights.mean(dim=0)
        token_weights[token2index[pad_token]] = 0
        self.aggregation = aggregation

        self.token_weights = nn.Parameter(token_weights)

    @classmethod
    def collate_fn(cls, batch):
        texts = [sample["text"] for sample in batch]
        token_ids = [sample["token_ids"] for sample in batch]

        max_len = max(len(sample) for sample in token_ids)
        token_ids = torch.stack(
            [F.pad(sample, (0, max_len - len(sample))) for sample in token_ids]
        )

        labels = [sample["label"] for sample in batch]

        return {
            "texts": texts,
            "token_ids": token_ids,
            "labels": torch.as_tensor(labels),
        }

    def forward(self, batch: Mapping[str, Any]) -> torch.Tensor:
        token_ids = batch["token_ids"]  # (batch_size, max_len)
        text_encodings = self.token_weights[
            token_ids
        ]  # (batch_size, max_len, space_dim)
        # now we could aggregate the encodings for each text, to get a single encoding for each one...
        if self.aggregation == "mean":
            # ...a way to do this is to sum the encodings and divide by the number of (non-padding) tokens
            mask = token_ids != self.token2index[PAD_TOKEN_STR]
            text_lengths = mask.sum(dim=1).unsqueeze(1).float()
            text_encodings = text_encodings.sum(dim=1) / text_lengths
        elif self.aggregation == None:
            # ...or we could just return the encodings for each token
            pass
        else:
            raise ValueError(
                f"Invalid aggregation: {self.aggregation}. Are you implementing a new one? :]"
            )
        return text_encodings

In [ ]:
encoder = TextEncoder(token2index=vocab, encoding_dim=encoding_dim)

In [ ]:
from torch.utils.data import DataLoader
import lightning as pl
from torchmetrics import MetricCollection, Accuracy

In [ ]:
class SimpleClassifier(pl.LightningModule):
    def __init__(self, encoder: nn.Module, encoding_dim: int, n_classes: int):
        super().__init__()
        self.encoder = encoder
        # we will use a linear probe to classify the encoded texts
        self.classifier = nn.Linear(encoding_dim, n_classes)

        # some metrics
        self.train_metrics = MetricCollection(
            {"accuracy": Accuracy(task="multiclass", num_classes=n_classes)}
        )
        self.val_metrics = self.train_metrics.clone()
        self.test_metrics = self.train_metrics.clone()

    def forward(self, texts: Sequence[Sequence[str]]) -> torch.Tensor:
        # get a single encoding for each text
        encoded = self.encoder(texts)
        # and then classify it with the linear probe
        return self.classifier(encoded)

    def _step(self, batch, batch_idx, stage: str):
        logits = self(batch)
        loss = F.cross_entropy(logits, batch["labels"].to(self.device))
        self.log(f"{stage}_loss", loss, on_step=True, on_epoch=True)
        self.log_dict(
            getattr(self, f"{stage}_metrics")(logits, batch["labels"]),
            on_step=False,
            on_epoch=True,
        )
        return loss

    def training_step(self, batch, batch_idx):
        return self._step(batch, batch_idx, stage="train")

    def validation_step(self, batch, batch_idx):
        return self._step(batch, batch_idx, stage="val")

    def test_step(self, batch, batch_idx):
        return self._step(batch, batch_idx, stage="test")

    def configure_optimizers(self):
        # Adam-driven deep learning
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [ ]:
model = SimpleClassifier(
    encoder,
    encoding_dim=encoding_dim,
    n_classes=train_data.features["label"].num_classes,
)

In [ ]:
model.eval()
sentence = "This movie is great, the best I've seen!"
tokenized = simple_tokenize(sentence)
with torch.no_grad():
  token_ids = torch.as_tensor(
      [vocab.get(token, vocab[UNK_TOKEN_STR]) for token in tokenized]
  )
  logits = model.classifier(model.encoder({"token_ids": token_ids[None]}))
  probs = logits.softmax(dim=-1)
  print(
        f"The class is {train_data.features['label'].int2str(probs.argmax().item())} with a probability of {probs.max().item():.2%}"
  )

The class is neg with a probability of 54.33%


In [ ]:
def train_model(model, collate_fn, max_epochs: int):
    num_workers: int = 8
    train_loader = DataLoader(
        data["train"],
        batch_size=256,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=num_workers,
        pin_memory=True,
    )
    val_loader = DataLoader(
        data["val"],
        batch_size=256,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=num_workers,
        pin_memory=True,
    )
    test_loader = DataLoader(
        data["test"],
        batch_size=256,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=num_workers,
        pin_memory=True,
    )

    trainer = pl.Trainer(max_epochs=max_epochs)
    trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)
    trainer.test(model=model, dataloaders=test_loader)

In [ ]:
train_model(model=model, collate_fn=TextEncoder.collate_fn, max_epochs=3)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUD

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=3` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.7901999950408936     │
│      test_loss_epoch      │     0.514161229133606     │
└───────────────────────────┴───────────────────────────┘

In [ ]:
model.eval()
sentence = "This movie is great, the best I've seen!"
tokenized = simple_tokenize(sentence)
with torch.no_grad():
  token_ids = torch.as_tensor(
      [vocab.get(token, vocab[UNK_TOKEN_STR]) for token in tokenized]
  )
  logits = model.classifier(model.encoder({"token_ids": token_ids[None]}))
  probs = logits.softmax(dim=-1)
  print(
        f"The class is {train_data.features['label'].int2str(probs.argmax().item())} with a probability of {probs.max().item():.2%}"
  )

The class is pos with a probability of 84.99%


In [ ]:
# the text encoder is so simple we can directly apply the classifier on top of the word encodings
token_scores = model.classifier.cpu()(model.encoder.token_weights.detach().cpu())
token_scores = token_scores.softmax(dim=-1)
token_scores.shape

torch.Size([10002, 2])

In [ ]:
index2token = {index: token for token, index in vocab.items()}

In [ ]:
# most_negative_tokens
most_negative_ids = (
    token_scores[:, 0].topk(20).indices.tolist()
)  # the first dimension corresponds to the positive class
most_negative_tokens = [index2token[index] for index in most_negative_ids]
most_negative_tokens

['unbelievable',
 'pathetic',
 'stupid',
 'waste',
 'thing',
 'bad',
 'gore',
 'minutes',
 'cannibals',
 'so',
 'yeah',
 'insult',
 'brain',
 'buried',
 'segal',
 'only',
 'acting',
 'slasher',
 'flick',
 'dreadful']

In [ ]:
# most_positive_tokens
most_positive_ids = (
    token_scores[:, 1].topk(20).indices.tolist()
)  # the second dimension corresponds to the negative class
most_positive_tokens = [index2token[index] for index in most_positive_ids]
most_positive_tokens

['profound',
 'and',
 'tears',
 'amazing',
 'invites',
 'daniel',
 'fun',
 'figures',
 'still',
 'rookie',
 'excellent',
 'spoil',
 'orchestra',
 'awareness',
 'collection',
 'mesmerizing',
 'dramas',
 'seeking',
 'restore',
 'best']

In [ ]:
class RNNEncoder(TextEncoder):
    def __init__(
        self,
        token2index: Mapping[str, int],
        encoding_dim: int,
        hidden_dim: int,
        oov_token: str = UNK_TOKEN_STR,
        pad_token: str = PAD_TOKEN_STR,
        rnn_type: str = "lstm",
    ):
        super().__init__(
            token2index, encoding_dim, oov_token, pad_token, aggregation="rnn"
        )
        rnn_type = rnn_type.lower()
        assert rnn_type in {"lstm", "gru"}
        self.rnn_type = rnn_type
        self.hidden_dim = hidden_dim
        self.rnn = getattr(nn, rnn_type.upper())(
            encoding_dim, hidden_dim, batch_first=True
        )

    def forward(self, batch: Mapping[str, Any]) -> torch.Tensor:
        token_ids = batch["token_ids"]
        mask = token_ids != self.token2index[PAD_TOKEN_STR]
        text_lengths = mask.sum(dim=1)

        token_encodings = self.token_weights[token_ids]
        packed_encodings = nn.utils.rnn.pack_padded_sequence(
            token_encodings, text_lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        packed_output, (hidden, cell) = self.rnn(packed_encodings)
        return hidden[-1]

In [ ]:
model = SimpleClassifier(
    RNNEncoder(token2index=vocab, encoding_dim=encoding_dim, hidden_dim=32),
    encoding_dim=32,
    n_classes=train_data.features["label"].num_classes,
)
model

SimpleClassifier(
  (encoder): RNNEncoder(
    (rnn): LSTM(100, 32, batch_first=True)
  )
  (classifier): Linear(in_features=32, out_features=2, bias=True)
  (train_metrics): MetricCollection(
    (accuracy): MulticlassAccuracy()
  )
  (val_metrics): MetricCollection(
    (accuracy): MulticlassAccuracy()
  )
  (test_metrics): MetricCollection(
    (accuracy): MulticlassAccuracy()
  )
)

In [ ]:
train_model(model=model, collate_fn=RNNEncoder.collate_fn, max_epochs=5)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name          | Type             | Params
---------------------------------------------------
0 | encoder       | RNNEncoder       | 1.0 M 
1 | classifier    | Linear           | 66    
2 | train_metrics | MetricCollection | 0     
3 | val_metrics   | MetricCollection | 0     
4 | test_metrics  | MetricCollection | 0     
----------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=5` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.8044800162315369     │
│      test_loss_epoch      │    0.4513998031616211     │
└───────────────────────────┴───────────────────────────┘